In [1]:
import numpy as np
import pandas as pd
from groq import Groq
import os
import time

In [2]:
df = pd.read_excel('../../data/raw/OMIEC_NEUROMORPHIC_07_08_24.xls')

In [3]:
GROQ_API_KEY='GROQ_API_KEY'

In [13]:
# groq
client = Groq(
    api_key=GROQ_API_KEY,
)

# respostas
responses_list = []

# todos os abstracts
total = len(df['Abstract'])
total_time = 0


for index, row in df.iterrows():
    start_time = time.time()

    # tempo
    print(f" {index} of {total}, remaining estimated time: {(total-index)*(total_time/(index+1))}")

    # chat request
    completion = client.chat.completions.create(
        model="llama3-8b-8192",
        messages=[
            {
                "role": "system",
                "content": "RolePlay as a bot seeking for polymers that are used in neuromorphic devices"
            },
            {
                "role": "user",
                #"content": "For the text I will input next, output only polymers used for neuromorphic devices. Do not output other types of polymers, and other kinds of materials, only the polymers. The output must be only the polymer material names separated by semicolons. Do not repeat the polymer name more than once. If no polymer was found, output 'None'"
                "content": "For the text I will input next, provide only the names of OMIEC polymers, also known as Organic Mixed Ionic Electronic Conductors, that are present in the text and are used for neuromorphic devices. Do not output other types of polymers, and other kinds of materials, only the OMIECs. If any polymer that meets the condition is found in the text, the output should be only the polymer material names separated by semicolons. If no polymer that meets the condition is found in the text, the output should be 'None'. Do not repeat the polymer name more than once. The output should not include the type of material. I want to use the output for future data analysis, so if the polymer name is written in full, replace it with its abbreviation."
            },
            {
                "role": "assistant",
                "content": "Hello, I will tell the polymers that are used in neuromorphic devices of any text you input next"
            },
            {
                "role": "user",
                "content": f"{row['Abstract']}"  # Use f-string to format the abstract
            }
        ],
        temperature=1,
        max_tokens=1024,
        top_p=1,
        stream=True,
        stop=None,
    )

    # coletar respostas
    response = ""

    # Stream the completion result and print the content
    for chunk in completion:
        response += chunk.choices[0].delta.content or ""
        print(chunk.choices[0].delta.content or "", end="")

    # listas de respostas
    responses_list.append({'index': index, 'response': response})

    # tempo total
    total_time += time.time() - start_time

 0 of 54, remaining estimated time: 0.0
PEDOT; PSS; PBTTT; PQR; F8BT 1 of 54, remaining estimated time: 10.378825068473816
PMIECs; PEO; PEO-I; PEOI 2 of 54, remaining estimated time: 12.441447575887045
PTCDA; MEH-PPV 3 of 54, remaining estimated time: 13.265033483505249
PEDOT;PSS 4 of 54, remaining estimated time: 14.432055950164797
P3HT 5 of 54, remaining estimated time: 19.526465217272442
Polymer; PIL 6 of 54, remaining estimated time: 19.087653023856028
PTTBT; P3HT 7 of 54, remaining estimated time: 18.50664696097374
gNR-Bu 8 of 54, remaining estimated time: 17.881032678816055
PEDOT; PSS/PEDOT 9 of 54, remaining estimated time: 17.29483437538147
Mel; PEDOT:PSS 10 of 54, remaining estimated time: 16.81930446624756
PMDT; MEH-PPV; P3HT; DAD-PPV; PEDOT; PSS-PV; PCELT; PHT-TIPS 11 of 54, remaining estimated time: 16.7414586742719
PEDOT; P3HT 12 of 54, remaining estimated time: 16.133092623490555
NDI-2Tz; P-XO (X = 3-6) 13 of 54, remaining estimated time: 15.733231050627571
PEDOT; PTB; po

In [5]:
# DataFrame
df_with_responses = pd.DataFrame(responses_list)

# salvar para .csv
df_with_responses.to_excel('../../dataframes/OMIEC_NEUROMORPHIC_RESPONSES_2.xlsx', index=False)

In [12]:
a = df['Abstract'][0]
print(a)

The field of organic mixed ionic-electronic conductors (OMIECs) has gained significant attention due to their ability to transport both electrons and ions, making them promising candidates for various applications. Initially focused on inorganic materials, the exploration of mixed conduction has expanded to organic materials, especially polymers, owing to their advantages such as solution processability, flexibility, and property tunability. OMIECs, particularly in the form of polymers, possess both electronic and ionic transport functionalities. This review provides an overview of OMIECs in various aspects covering mechanisms of charge transport including electronic transport, ionic transport, and ionic-electronic coupling, as well as conducting/semiconducting conjugated polymers and their applications in organic bioelectronics, including (multi)sensors, neuromorphic devices, and electrochromic devices. OMIECs show promise in organic bioelectronics due to their compatibility with biol

In [11]:
a

'Neuromorphic devices are likely to be the next evolution of computing, allowing to implement machine learning within hardware components. In biological neural systems, learning and signal processing are achieved by communication between neurons through time-dependent ion flux in the synapses. Integrating such ion-mediated operating principles in neuromorphic devices can deliver an energy efficient and powerful technology. Here a device known as a light-emitting electrochemical cell is revisited and modified, exploiting its ability to modulate current through ion accumulation/depletion at the electrodes and turn it into an organic synaptic diode. This two-terminal device is based on an organic mixed ionic-electronic conducting polymer that serves as active layer for conduction of lithium ions as well as charge carriers. The ionic conduction properties are modified by cryptand molecules, able to reversibly capture ions. The device can be reliably switched between states for at least 100

#### Playground

In [6]:
import requests
import time

df_with_responses = pd.DataFrame()

total = len(full_df['Abstract'])
total_time = 0

for index, row in full_df.iterrows():
    start_time = time.time()
    print(index, 'of', total, 'remaining estimated time', (total-index)*(total_time/(index+1)))
    payload = { 
        'prompt': """{
        'systemPrompt': 'RolePlay as a bot seeking for catalysts', 
        'user': "For the text I will input next, output only the catalyst materials used for the oxygen reduction reaction. Do not output precursors, electrolytes, and other kinds of materials, only the catalyst. The output must be only the catalyst material names separated by semicolons. Do not repeat the material name more than once. If no catalyst was found, output 'None'",
        'Assistant': 'Hello, I will tell the catalysts of any text you input next',
        'user_text':""" + row['Abstract'] + "}",
        "temperature":0.75,
        "topP":0.9,
        "maxTokens": 1000
    }
    #print(payload)
    response = requests.post('https://fumes-api.onrender.com/llama3', json=payload, stream=True)
    filtered_response = response.text.replace(' YOU CAN BUY ME COFFE! https://buymeacoffee.com/mygx', '')
    print(filtered_response)
    row['response'] = filtered_response
    df_with_responses = df_with_responses.append(row)
    time.sleep(2.5)
    total_time = total_time + time.time() - start_time
    print('Time for this loop:', time.time() - start_time, 'average loop time:', (total_time/(index+1)))
    #for chunk in response.iter_content(chunk_size=1024):  
    #    if chunk:
    #        print(chunk.decode('utf-8'))

NameError: name 'full_df' is not defined